# Contextual Bandits for Ranking with TF-Agents

> see [ranking tutorial](https://www.tensorflow.org/agents/tutorials/ranking_tutorial)

### Overview

* The contextual bandits approach is classified as an extension of multi-armed bandits
* a contextual multi-armed banded problem is a simplified reinforcement learning algorithm where the agent takes an action from a set of possible actions 

> **TODO**

The **Bandit Ranking** agent will be similar to the `NeuralEpsilonGreedy` agent. Main differences:

* The item features are stored in the `per_arm` part of the observation, in the order of how they are recommended
* Since this ordered list of items expresses what action was taken by the policy,
the `action` value of the trajectory is not used by the agent.

> Note: difference between the "per-arm" observation recieved by the policy vs the agent:

While the agent receives the items in the recommendation slots, the policy receives the items that are available for recommendation. The user is responsible for converting the observation to the
syntax required by the agent.


The training observation contains the global features and the features of the items in the recommendation slots 
* The item features are stored in the `per_arm` part of the observation, in the order of how they are recommended
* Note: since this ordered list of items expresses what action was taken by the policy, the action value of the trajectory is not used by the agent

## Load notebook config

* use the prefix defined in `00-env-setup`

In [1]:
VERSION        = "v1"                       # TODO
PREFIX         = f'rec-bandits-{VERSION}'   # TODO

# PREFIX = 'mabv1'

print(f"PREFIX: {PREFIX}")

PREFIX: rec-bandits-v1


In [2]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "rec-bandits-v1"
VERSION                  = "v1"

BUCKET_NAME              = "rec-bandits-v1-hybrid-vertex-bucket"
BUCKET_URI               = "gs://rec-bandits-v1-hybrid-vertex-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://rec-bandits-v1-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

VPC_NETWORK_FULL         = "projects/934903580331/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_ID      = "hybrid_vertex.movielens_ds_rec_bandits_v1"
BIGQUERY_TABLE_ID        = "hybrid_vertex.movielens_ds_rec_bandits_v1.training_dataset"

REPO

## imports

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
!pwd

/home/jupyter/tf_vertex_agents/03-ranking


In [5]:
import functools
from collections import defaultdict
from typing import Callable, Dict, List, Optional, TypeVar, Iterable
from datetime import datetime
import time
from pprint import pprint
import pickle as pkl
import pandas as pd

from tqdm import tqdm

# logging
import logging
logging.disable(logging.WARNING)

import matplotlib.pyplot as plt
import numpy as np

# google cloud
from google.cloud import aiplatform, storage

# tensorflow
import tensorflow as tf
# from tf_agents.agents import TFAgent

# from tf_agents.bandits.environments import stationary_stochastic_per_arm_py_environment as p_a_env
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
# from tf_agents.drivers import dynamic_step_driver
# from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts

# from tf_agents.bandits.agents import lin_ucb_agent
# from tf_agents.bandits.agents import linear_thompson_sampling_agent as lin_ts_agent
from tf_agents.bandits.agents import neural_epsilon_greedy_agent
from tf_agents.bandits.agents import neural_linucb_agent
from tf_agents.bandits.networks import global_and_arm_feature_network
from tf_agents.bandits.policies import policy_utilities

from tf_agents.bandits.specs import utils as bandit_spec_utils
from tf_agents.trajectories import trajectory
from tf_agents.bandits.environments import ranking_environment
from tf_agents.bandits.agents import ranking_agent

# GPU
from numba import cuda 
import gc

import sys
sys.path.append("..")

# this repo
from src.per_arm_rl import data_utils
from src.per_arm_rl import data_config
from src.per_arm_rl import train_utils as train_utils

# tf exceptions and vars
if tf.__version__[0] != "2":
    raise Exception("The trainer only runs with TensorFlow version 2.")

T = TypeVar("T")

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [7]:
device = cuda.get_current_device()
device.reset()
gc.collect()

14

In [8]:
# cloud storage client
storage_client = storage.Client(project=PROJECT_ID)

# Vertex client
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Data prep

In [10]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.AUTO

In [11]:
NUM_EXAMPLES_PER_LIST = 5 # 3 | 5

# SPLIT = "val"
# SPLIT = "listwise-val"
SPLIT = f"listwise-{NUM_EXAMPLES_PER_LIST}n-val"

print(f"SPLIT: {SPLIT}")

SPLIT: listwise-5n-val


In [12]:
! gsutil ls $BUCKET_URI/$DATA_GCS_PREFIX/$SPLIT/

gs://rec-bandits-v1-hybrid-vertex-bucket/data/listwise-5n-val/ml-100k-listwise-5n-val.tfrecord


### Val

In [13]:
SPLIT = "val"
# SPLIT = "listwise-val"
SPLIT = f"listwise-{NUM_EXAMPLES_PER_LIST}n-val"

val_files = []
for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_GCS_PREFIX}/{SPLIT}'):
    if '.tfrecord' in blob.name:
        val_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
        
val_dataset = tf.data.TFRecordDataset(val_files)
val_dataset = val_dataset.map(data_utils.parse_lw_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)

for example in val_dataset.take(1):
    pprint(example)

{'movie_genres': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 3,  0,  0,  0, 19])>,
 'movie_id': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'94', b'245', b'403', b'50', b'470'], dtype=object)>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'346'>,
 'user_rating': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([3., 4., 3., 5., 3.], dtype=float32)>}


### Train

In [14]:
# SPLIT = "train"
# SPLIT = "listwise-train"
# SPLIT = f"listwise-{NUM_EXAMPLES_PER_LIST}n-train"

# train_files = []
# for blob in storage_client.list_blobs(f"{BUCKET_NAME}", prefix=f'{DATA_GCS_PREFIX}/{SPLIT}'):
#     if '.tfrecord' in blob.name:
#         train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
        
# train_dataset = tf.data.TFRecordDataset(train_files)
# train_dataset = train_dataset.map(data_utils.parse_lw_tfrecord)

# for x in train_dataset.batch(1).take(1):
#     pprint(x)

In [15]:
train_dataset = val_dataset

### Vocab

In [16]:
GENERATE_VOCABS = False

print(f"GENERATE_VOCABS: {GENERATE_VOCABS}")

GENERATE_VOCABS: False


In [17]:
EXISTING_VOCAB_FILE = f'gs://{BUCKET_NAME}/{VOCAB_SUBDIR}/{VOCAB_FILENAME}'

!gsutil cp gs://mabv1-hybrid-vertex-bucket/vocabs/vocab_dict.pkl $EXISTING_VOCAB_FILE

Copying gs://mabv1-hybrid-vertex-bucket/vocabs/vocab_dict.pkl [Content-Type=application/octet-stream]...
/ [1 files][142.9 KiB/142.9 KiB]                                                
Operation completed over 1 objects/142.9 KiB.                                    


In [18]:
data_utils.download_blob(
    project_id = PROJECT_ID,
    bucket_name = BUCKET_NAME, 
    source_blob_name = f"{VOCAB_SUBDIR}/{VOCAB_FILENAME}", 
    destination_file_name= VOCAB_FILENAME
)

filehandler = open(VOCAB_FILENAME, 'rb')
vocab_dict = pkl.load(filehandler)
filehandler.close()

for key in vocab_dict.keys():
    pprint(key)

Downloaded storage object vocabs/vocab_dict.pkl from bucket rec-bandits-v1-hybrid-vertex-bucket to local file vocab_dict.pkl.
'movie_id'
'user_id'
'user_occupation_text'
'movie_genres'
'bucketized_user_age'
'max_timestamp'
'min_timestamp'
'timestamp_buckets'


# Global & Per-Arm feature embedding models 

In [19]:
NUM_OOV_BUCKETS       = 2
GLOBAL_EMBEDDING_SIZE = 64
MV_EMBEDDING_SIZE     = 32 #32

BATCH_SIZE            = 5 #128
EVAL_BATCH_SIZE       = 1

NUM_ITEMS             = NUM_EXAMPLES_PER_LIST # 3 | 5 
NUM_SLOTS             = 2

DISTANCE_THRESHOLD    = 0.5

print(f"NUM_OOV_BUCKETS    : {NUM_OOV_BUCKETS}")
print(f"GLOBAL_EMB_SIZE    : {GLOBAL_EMBEDDING_SIZE}")
print(f"MV_EMB_SIZE        : {MV_EMBEDDING_SIZE}")
print(f"BATCH_SIZE         : {BATCH_SIZE}")
print(f"EVAL_BATCH_SIZE    : {EVAL_BATCH_SIZE}")
print(f"NUM_ITEMS          : {NUM_ITEMS}")
print(f"NUM_SLOTS          : {NUM_SLOTS}")
print(f"DISTANCE_THRESHOLD : {DISTANCE_THRESHOLD}")

NUM_OOV_BUCKETS    : 2
GLOBAL_EMB_SIZE    : 64
MV_EMB_SIZE        : 32
BATCH_SIZE         : 5
EVAL_BATCH_SIZE    : 1
NUM_ITEMS          : 5
NUM_SLOTS          : 2
DISTANCE_THRESHOLD : 0.5


In [20]:
for i in range(1):
    
    iterator = iter(train_dataset.batch(1))
    data = next(iterator)

data

{'movie_genres': <tf.Tensor: shape=(1, 5), dtype=int64, numpy=array([[ 3,  0,  0,  0, 19]])>,
 'movie_id': <tf.Tensor: shape=(1, 5), dtype=string, numpy=array([[b'94', b'245', b'403', b'50', b'470']], dtype=object)>,
 'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'346'], dtype=object)>,
 'user_rating': <tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[3., 4., 3., 5., 3.]], dtype=float32)>}

## Embedding layers

#### User ID

In [21]:
user_id_input_layer = tf.keras.Input(
    name="user_id",
    shape=(1,),
    dtype=tf.string
)

user_id_lookup = tf.keras.layers.StringLookup(
    max_tokens=len(vocab_dict['user_id']) + NUM_OOV_BUCKETS,
    num_oov_indices=NUM_OOV_BUCKETS,
    mask_token=None,
    vocabulary=vocab_dict['user_id'],
)(user_id_input_layer)

user_id_embedding = tf.keras.layers.Embedding(
    # Let's use the explicit vocabulary lookup.
    input_dim=len(vocab_dict['user_id']) + NUM_OOV_BUCKETS,
    output_dim=GLOBAL_EMBEDDING_SIZE
)(user_id_lookup)

# user_id_embedding = tf.reduce_sum(user_id_embedding, axis=-2)

In [22]:
test_user_id_model = tf.keras.Model(inputs=user_id_input_layer, outputs=user_id_embedding)

for x in train_dataset.batch(1).take(1):
    print(x["user_id"])
    print(test_user_id_model(x["user_id"]))

tf.Tensor([b'346'], shape=(1,), dtype=string)
tf.Tensor(
[[[-0.00946199 -0.00779102  0.02253656 -0.0281036  -0.04907619
   -0.00111315  0.01590214 -0.01033651 -0.02272823 -0.03619151
   -0.02710731 -0.02966145 -0.00131334 -0.0435203  -0.0052826
   -0.03779681 -0.04004806 -0.04469528  0.03358519 -0.03440194
    0.0200894   0.0077446   0.04805661  0.04013901  0.02096145
   -0.00709069 -0.00105106  0.00118973  0.03069612 -0.0464071
    0.03246001  0.04133833 -0.03127392  0.00970823  0.04900965
    0.01276049 -0.02910261  0.04109125  0.02205325 -0.04546207
   -0.02464928  0.04513765 -0.0467082   0.04213704  0.04484126
    0.01052315  0.04005774 -0.00830711  0.00455688  0.02888402
    0.04418704 -0.02264543  0.04993678 -0.02518172 -0.00902994
    0.00110137 -0.04690304 -0.0430104  -0.02446631 -0.04867182
   -0.01480299  0.02161438  0.03757143  0.00906471]]], shape=(1, 1, 64), dtype=float32)


#### Movie ID

In [23]:
len(vocab_dict['movie_id'])

1683

In [24]:
mv_id_input_layer = tf.keras.Input(
    name="movie_id",
    shape=(1,),
    dtype=tf.string
)

mv_id_lookup = tf.keras.layers.StringLookup(
    max_tokens=len(vocab_dict['movie_id']) + NUM_OOV_BUCKETS,
    num_oov_indices=NUM_OOV_BUCKETS,
    mask_token=None,
    vocabulary=vocab_dict['movie_id'],
)(mv_id_input_layer)

mv_id_embedding = tf.keras.layers.Embedding(
    # Let's use the explicit vocabulary lookup.
    input_dim=len(vocab_dict['movie_id']) + NUM_OOV_BUCKETS,
    output_dim=MV_EMBEDDING_SIZE
)(mv_id_lookup)

# mv_id_embedding = tf.reduce_sum(mv_id_embedding, axis=-2)

In [25]:
test_mv_id_model = tf.keras.Model(inputs=mv_id_input_layer, outputs=mv_id_embedding)

for x in train_dataset.batch(1).take(1):
    print(x["movie_id"])
    list_length = x["movie_id"].shape[1]
    print(test_mv_id_model(x["movie_id"]))

tf.Tensor([[b'94' b'245' b'403' b'50' b'470']], shape=(1, 5), dtype=string)
tf.Tensor(
[[[-0.01636115 -0.04689224 -0.013713    0.02641647  0.0194614
    0.04733548  0.03533632 -0.00333887  0.02818942  0.02723351
    0.01625511  0.03942041  0.00359466  0.01963184 -0.03519678
   -0.00671798 -0.0414732  -0.03343675  0.00793576  0.02577437
    0.04504141 -0.03399416  0.02853381 -0.03645446  0.01590827
    0.00858242  0.04317639  0.0464666  -0.02829629 -0.04383949
    0.04908258  0.00221925]
  [ 0.04584137  0.00485065 -0.00277252  0.04626917 -0.04966178
    0.04113512  0.00662166 -0.0210382   0.00072246  0.0453555
   -0.03763857 -0.00435748  0.00706385 -0.01044433  0.01312664
    0.03961308 -0.03077385  0.04214424 -0.02181128 -0.00564315
    0.0445034   0.03317592  0.00708282 -0.00159972  0.04386225
   -0.0009255   0.04063422  0.02419226  0.00123237  0.0248446
   -0.02588284 -0.03315727]
  [-0.04244614  0.01901117  0.01638852  0.02880948 -0.03012358
    0.00730245 -0.00808992  0.04261055  0

In [26]:
test_mv_id_model = tf.keras.Model(inputs=mv_id_input_layer, outputs=mv_id_embedding)

for x in train_dataset.batch(1).take(1):
    print(x["movie_id"])
    list_length = x["movie_id"].shape[1]
    print(test_mv_id_model(x["movie_id"]))

tf.Tensor([[b'94' b'245' b'403' b'50' b'470']], shape=(1, 5), dtype=string)
tf.Tensor(
[[[-0.01636115 -0.04689224 -0.013713    0.02641647  0.0194614
    0.04733548  0.03533632 -0.00333887  0.02818942  0.02723351
    0.01625511  0.03942041  0.00359466  0.01963184 -0.03519678
   -0.00671798 -0.0414732  -0.03343675  0.00793576  0.02577437
    0.04504141 -0.03399416  0.02853381 -0.03645446  0.01590827
    0.00858242  0.04317639  0.0464666  -0.02829629 -0.04383949
    0.04908258  0.00221925]
  [ 0.04584137  0.00485065 -0.00277252  0.04626917 -0.04966178
    0.04113512  0.00662166 -0.0210382   0.00072246  0.0453555
   -0.03763857 -0.00435748  0.00706385 -0.01044433  0.01312664
    0.03961308 -0.03077385  0.04214424 -0.02181128 -0.00564315
    0.0445034   0.03317592  0.00708282 -0.00159972  0.04386225
   -0.0009255   0.04063422  0.02419226  0.00123237  0.0248446
   -0.02588284 -0.03315727]
  [-0.04244614  0.01901117  0.01638852  0.02880948 -0.03012358
    0.00730245 -0.00808992  0.04261055  0

In [27]:
list_length

5

In [28]:
for x in train_dataset.batch(1).take(1):
    print(x["movie_id"])
    single_sample = x["movie_id"][0]
    item_1 = tf.gather(x["movie_id"][0], 0)
    print(item_1)

tf.Tensor([[b'94' b'245' b'403' b'50' b'470']], shape=(1, 5), dtype=string)
tf.Tensor(b'94', shape=(), dtype=string)


In [29]:
single_sample[0]

<tf.Tensor: shape=(), dtype=string, numpy=b'94'>

#### Movie Genres

In [30]:
len(vocab_dict['movie_genres'])

20

In [31]:
mv_genre_input_layer = tf.keras.Input(
    name="movie_genres",
    shape=(1,),
    dtype=tf.float32
)

mv_genre_lookup = tf.keras.layers.IntegerLookup(
    vocabulary=vocab_dict['movie_genres'],
    num_oov_indices=NUM_OOV_BUCKETS,
    oov_value=0,
)(mv_genre_input_layer)

mv_genre_embedding = tf.keras.layers.Embedding(
    # Let's use the explicit vocabulary lookup.
    input_dim=len(vocab_dict['movie_genres']) + NUM_OOV_BUCKETS,
    output_dim=MV_EMBEDDING_SIZE
)(mv_genre_lookup)

# mv_genre_embedding = tf.reduce_sum(mv_genre_embedding, axis=-2)

In [32]:
test_mv_gen_model = tf.keras.Model(inputs=mv_genre_input_layer, outputs=mv_genre_embedding)

for x in train_dataset.batch(1).take(1):
    print(x["movie_genres"])
    print(test_mv_gen_model(x["movie_genres"]))

tf.Tensor([[ 3  0  0  0 19]], shape=(1, 5), dtype=int64)
tf.Tensor(
[[[ 2.3156811e-02  1.3729598e-02 -3.7662707e-02  1.5712153e-02
    7.7462420e-03  8.3059892e-03  1.4813986e-02 -3.0442357e-02
    4.0788677e-02 -2.4301970e-02 -1.7502319e-02  2.7880523e-02
   -4.4727266e-02  5.2415729e-03  9.1277249e-03 -2.7250314e-02
    4.7937755e-02  2.5181398e-03 -8.1559271e-04  4.5005251e-02
   -4.2149793e-02  3.2151666e-02  2.5014345e-02 -1.8513452e-02
    2.0792056e-02 -2.4676992e-02 -4.9909770e-02  2.4801467e-02
    3.0246112e-02  6.7820661e-03  3.1125795e-02 -2.4241043e-02]
  [-2.1569444e-02 -1.1244036e-02  4.6417009e-02 -3.7423253e-02
    4.7383953e-02 -9.3461387e-03  4.0958151e-03 -1.9301726e-02
    2.1182824e-02  1.2697231e-02  9.1457143e-03  4.3180738e-02
    1.6883980e-02  3.6405336e-02 -1.2547530e-02 -4.8290838e-02
   -4.3536045e-02  4.2986125e-05 -3.3918679e-02 -1.9695951e-02
    1.6248491e-02 -3.5038937e-02 -7.7270754e-03 -1.5462540e-02
    1.0850299e-02 -4.8117749e-03  4.1247036e-02  

In [33]:
for x in train_dataset.batch(1).take(1):
    print(x["movie_genres"])
    mv_gen_sample = x["movie_genres"]
    item_gen = tf.gather(x["movie_genres"], 0)
    print(item_gen)

tf.Tensor([[ 3  0  0  0 19]], shape=(1, 5), dtype=int64)
tf.Tensor([ 3  0  0  0 19], shape=(5,), dtype=int64)


In [34]:
for x in train_dataset.batch(1).take(1):
    print(x["user_rating"])
    # mv_gen_sample = x["movie_genres"]
    # item_gen = tf.gather(x["movie_genres"], 0)
    # print(item_gen)

tf.Tensor([[3. 4. 3. 5. 3.]], shape=(1, 5), dtype=float32)


## Sampling Functions

#### item sampling

In [36]:
# def _get_per_arm_features(x):
#     """
#     This function generates a single per-arm observation vector.
#     """
    
#     id_1 = tf.gather(x["movie_id"][0], 0)
#     id_2 = tf.gather(x["movie_id"][0], 1)
#     id_3 = tf.gather(x["movie_id"][0], 2)
#     # print(f"id_1 shape: {id_1.shape}")
    
#     stacked_ids = tf.stack(
#         [
#             test_mv_id_model(id_1), 
#             test_mv_id_model(id_2), 
#             test_mv_id_model(id_3)
#         ], axis = 0
#     )
#     # print(f"stacked_ids shape: {stacked_ids.shape}")
#     # print(f"stacked_ids: {stacked_ids}")
        
    
#     gen_1 = tf.gather(x["movie_genres"][0], 0)
#     gen_2 = tf.gather(x["movie_genres"][0], 1)
#     gen_3 = tf.gather(x["movie_genres"][0], 2)
#     # print(f"gen_1 shape: {gen_1.shape}")
    
#     stacked_gens = tf.stack(
#         [
#             test_mv_gen_model(gen_1), 
#             test_mv_gen_model(gen_2), 
#             test_mv_gen_model(gen_3)
#         ], axis = 0
#     )
#     # print(f"stacked_gens shape: {stacked_gens.shape}")
#     # print(f"stacked_gens: {stacked_gens}")
    
#     ratings_list = []

    
#     concat_embeddings = tf.concat(
#         [stacked_ids, stacked_gens], axis=1
#     )
#     # print(f"concat_embeddings shape: {concat_embeddings.shape}")
    
#     return concat_embeddings

In [38]:
def _get_per_arm_features(x):
    """
    This function generates a single per-arm observation vector.
    """
    
    # user rating
    ratings_list = x["user_rating"] #[0]
    indices = tf.argsort(ratings_list, direction="DESCENDING")
    
    # arm features
    arm_ids = []
    arm_gens = []
    
    for i in range(0, NUM_EXAMPLES_PER_LIST):
        arm_ids.append(test_mv_id_model(tf.gather(x["movie_id"][0], i)))
        # arm_ids.append(test_mv_id_model(tf.gather(x["movie_id"], i)))
        
    stacked_ids = tf.stack(arm_ids, axis = 0)
    
    for i in range(0, NUM_EXAMPLES_PER_LIST):
        arm_gens.append(test_mv_gen_model(tf.gather(x["movie_genres"][0], i)))
        # arm_gens.append(test_mv_gen_model(tf.gather(x["movie_genres"], i)))
        
    stacked_gens = tf.stack(arm_gens, axis = 0)
    
    concat_embeddings = tf.concat(
        [stacked_ids, stacked_gens], axis=1
    )
    print(f"concat_embeddings shape: {concat_embeddings.shape}")
    
    ordered_concat = tf.gather(concat_embeddings, indices, batch_dims=0) #.numpy()
    ordered_concat_embeddings = tf.stack(ordered_concat, axis = 0)
    
    # return concat_embeddings
    return ordered_concat_embeddings

In [39]:
# def return_ranking_rewards(x):
    
#     # ratings_list = []
    
#     # for i in range(0, NUM_EXAMPLES_PER_LIST):
#         # ratings_list.append(tf.gather(x["user_rating"][0], i))
#         # ratings_list.append(x["user_rating"][0])
    
#     ratings_list = x["user_rating"][0]
#     print(f"ratings_list: {ratings_list}")
    
#     indices = tf.argsort(ratings_list, direction="DESCENDING")
#     print(f"indices: {indices}")
    
#     ordered_list = tf.gather(ratings_list, indices, batch_dims=-1) #.numpy()
#     print(f"ordered_list: {ordered_list}")
    
#     return ordered_list

    
# ratings_list = return_ranking_rewards(data)
# ratings_list

In [40]:
test_arms = _get_per_arm_features(data)

PER_ARM_DIM = test_arms.shape[1]            
print(f"PER_ARM_DIM: {PER_ARM_DIM}")

test_arms

concat_embeddings shape: (5, 64)
PER_ARM_DIM: 5


<tf.Tensor: shape=(1, 5, 64), dtype=float32, numpy=
array([[[-1.26273520e-02,  4.15934660e-02, -2.60733850e-02,
         -8.38690996e-03, -1.09513626e-02,  4.70070355e-02,
         -4.77330685e-02, -3.63631137e-02,  4.27447259e-04,
          3.84168737e-02, -3.51316333e-02, -4.16781902e-02,
         -1.80354603e-02,  4.93993424e-02, -2.31365692e-02,
          2.39040144e-02, -4.20735851e-02, -9.94621590e-03,
         -9.27622244e-03, -3.94237153e-02,  1.90847032e-02,
         -1.22569911e-02, -3.77957895e-03, -7.68158585e-03,
          1.58616155e-03,  3.28605287e-02,  2.30605640e-02,
          3.82040180e-02,  7.47006014e-03, -4.12951782e-03,
          3.08122896e-02, -2.27085482e-02, -2.15694439e-02,
         -1.12440363e-02,  4.64170091e-02, -3.74232531e-02,
          4.73839529e-02, -9.34613869e-03,  4.09581512e-03,
         -1.93017256e-02,  2.11828239e-02,  1.26972310e-02,
          9.14571434e-03,  4.31807376e-02,  1.68839805e-02,
          3.64053361e-02, -1.25475302e-02, -4.82

#### global sampling

In [41]:
def _get_global_context_features(x):
    """
    This function generates a single global observation vector.
    """
    # _id_1 = test_user_id_model(x['user_id'])
    
    return test_user_id_model(x['user_id'])
    
    # list_length = x["movie_id"].shape[1]

In [47]:
test_globals = _get_global_context_features(data)

GLOBAL_DIM = test_globals.shape[2] 
print(f"GLOBAL_DIM: {GLOBAL_DIM}")

test_globals

GLOBAL_DIM: 64


<tf.Tensor: shape=(1, 1, 64), dtype=float32, numpy=
array([[[-0.00946199, -0.00779102,  0.02253656, -0.0281036 ,
         -0.04907619, -0.00111315,  0.01590214, -0.01033651,
         -0.02272823, -0.03619151, -0.02710731, -0.02966145,
         -0.00131334, -0.0435203 , -0.0052826 , -0.03779681,
         -0.04004806, -0.04469528,  0.03358519, -0.03440194,
          0.0200894 ,  0.0077446 ,  0.04805661,  0.04013901,
          0.02096145, -0.00709069, -0.00105106,  0.00118973,
          0.03069612, -0.0464071 ,  0.03246001,  0.04133833,
         -0.03127392,  0.00970823,  0.04900965,  0.01276049,
         -0.02910261,  0.04109125,  0.02205325, -0.04546207,
         -0.02464928,  0.04513765, -0.0467082 ,  0.04213704,
          0.04484126,  0.01052315,  0.04005774, -0.00830711,
          0.00455688,  0.02888402,  0.04418704, -0.02264543,
          0.04993678, -0.02518172, -0.00902994,  0.00110137,
         -0.04690304, -0.0430104 , -0.02446631, -0.04867182,
         -0.01480299,  0.02161438

# Ranking Agent

## Feedback type

Ranking agents assume either a `score_vector` or `cascading feedback` framework for the feedback signal (reward). 

* `score_vector`: feedback is a vector of scores for every item in the slots. 
* `cascading feedback`: if the kth item was clicked, then the items up to k-1 receive a score of -1, the kth item receives a score based on a feedback value, while the rest of the items receive feedback of 0. 

Ranking agent objective: train the scoring network to be able to estimate the above scores

In [48]:
# feedback_model = ranking_environment.FeedbackModel.CASCADING
feedback_model = ranking_environment.FeedbackModel.SCORE_VECTOR

# feedback_model

## Tensor Specs

example Tensor Spec structures...

`observation_spec()`

```python
{'global': TensorSpec(shape=(9,), dtype=tf.float32, name=None),
 'per_arm': TensorSpec(shape=(50, 11), dtype=tf.float32, name=None)}
```

`action_spec()`

```python
BoundedTensorSpec(shape=(3,), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(49, dtype=int32))
```

`reward_spec()`

```python
{'chosen_index': BoundedTensorSpec(shape=(), dtype=tf.int32, name='chosen_index', minimum=array(0, dtype=int32), maximum=array(3, dtype=int32)),
 'chosen_value': TensorSpec(shape=(), dtype=tf.float32, name='chosen_value')}
```

`time_step_spec()`

```python
TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': {'global': TensorSpec(shape=(9,), dtype=tf.float32, name=None),
                 'per_arm': TensorSpec(shape=(50, 11), dtype=tf.float32, name=None)},
 'reward': {'chosen_index': BoundedTensorSpec(shape=(), dtype=tf.int32, name='chosen_index', minimum=array(0, dtype=int32), maximum=array(3, dtype=int32)),
            'chosen_value': TensorSpec(shape=(), dtype=tf.float32, name='chosen_value')},
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})
```

set vars

In [49]:
from tf_agents.specs import array_spec

print(f"BATCH_SIZE         : {BATCH_SIZE}")
print(f"EVAL_BATCH_SIZE    : {EVAL_BATCH_SIZE}")
print(f"NUM_ITEMS          : {NUM_ITEMS}")
print(f"NUM_SLOTS          : {NUM_SLOTS}")
print(f"DISTANCE_THRESHOLD : {DISTANCE_THRESHOLD}")
print(f"GLOBAL_DIM         : {GLOBAL_DIM}")
print(f"PER_ARM_DIM        : {PER_ARM_DIM}")

BATCH_SIZE         : 5
EVAL_BATCH_SIZE    : 1
NUM_ITEMS          : 5
NUM_SLOTS          : 2
DISTANCE_THRESHOLD : 0.5
GLOBAL_DIM         : 64
PER_ARM_DIM        : 5


### Observation spec

In [50]:
observation_spec = {
    'global': tf.TensorSpec([GLOBAL_DIM], tf.float32),
    'per_arm': tf.TensorSpec([NUM_ITEMS, PER_ARM_DIM], tf.float32) #excluding action dim here
}
observation_spec

{'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None),
 'per_arm': TensorSpec(shape=(5, 5), dtype=tf.float32, name=None)}

### Action spec

> Action spec for ranking models must have rank 1

In [51]:
action_spec = tensor_spec.BoundedTensorSpec(
    shape=(NUM_SLOTS,),
    dtype=tf.int32,
    minimum=tf.constant(0),            
    maximum=NUM_ITEMS-1, # n degrees of freedom and will dictate the expected mean reward spec shape
    name="action_spec"
)

print(f"action_spec rank: {action_spec.shape.rank}")

action_spec

action_spec rank: 1


BoundedTensorSpec(shape=(2,), dtype=tf.int32, name='action_spec', minimum=array(0, dtype=int32), maximum=array(4, dtype=int32))

### Reward spec

In [52]:
if feedback_model == ranking_environment.FeedbackModel.CASCADING:
    # `chosen_index == num_slots` means no recommended item was clicked.
    reward_spec = {
        'chosen_index': array_spec.BoundedArraySpec(
            shape=[],
            minimum=0,
            maximum=NUM_SLOTS,
            dtype=np.int32,
            name='chosen_index',
        ),
        'chosen_value': array_spec.ArraySpec(
            shape=[], dtype=np.float32, name='chosen_value'
        ),
    }
elif feedback_model == ranking_environment.FeedbackModel.SCORE_VECTOR:
    reward_spec = array_spec.ArraySpec(
        shape=[NUM_SLOTS], dtype=np.float32, name='score_vector'
    )
else:
    reward_spec = f"Feedback model: {feedback_model}, not implemented"
    
reward_spec

ArraySpec(shape=(2,), dtype=dtype('float32'), name='score_vector')

### TimeStep spec

In [53]:
time_step_spec = ts.time_step_spec(
    observation_spec = observation_spec, 
    # reward_spec = reward_spec
)
time_step_spec

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': {'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None),
                 'per_arm': TensorSpec(shape=(5, 5), dtype=tf.float32, name=None)},
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

Inspect chosen arm features spec

In [54]:
time_step_spec.observation

{'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None),
 'per_arm': TensorSpec(shape=(5, 5), dtype=tf.float32, name=None)}

## Policy and Scoring Network

> all ranking agents train a network that estimates scores of item/user pairs

**Ranking Policies**
* `DESCENDING_SCORES` - Stack rank deterministically by scores
* `NO_PENALTY` - Sampling sequentially based on scores; no penalty applied
* `COSINE_DISTANCE` - Sampling sequentally and taking diversity into account

`penalty_mixture` parameter governs the balance between ranking based on scores and accounting for diversity
* low positive value --> ranking has less diversity
* higher value --> enforces more diversity

`logits_temperature` - temperature parameter for non-deterministic policies
* This value must be positive

In [55]:
AGENT_TYPE = "Ranking"
NETWORK_TYPE = "dotproduct"
POLICY_TYPE = ranking_agent.RankingPolicyType.COSINE_DISTANCE # COSINE_DISTANCE | NO_PENALTY | DESCENDING_SCORES

PENALTY_MIXTURE = 1.0
LOGITS_TEMPERATURE = 1.0

LEARNING_RATE = 0.005

GLOBAL_LAYERS   = [64, 32, 16]
ARM_LAYERS      = [64, 32, 16]
COMMON_LAYERS   = [16, 8]

HPARAMS = {  # TODO - streamline and consolidate
    "batch_size": BATCH_SIZE,
    "eval_batch_size" : EVAL_BATCH_SIZE,
    "num_itmes": NUM_ITEMS,
    "num_slots": NUM_SLOTS,
    "model_type": AGENT_TYPE,
    "network_type": NETWORK_TYPE,
    "global_layers": GLOBAL_LAYERS,
    "per_arm_layers": ARM_LAYERS,
    "common_layers": COMMON_LAYERS,
    "learning_rate": LEARNING_RATE,
    "policy_type": POLICY_TYPE,
    "feedback_model" : feedback_model,
    "penalty_mixture": PENALTY_MIXTURE,
    "logits_temperature": LOGITS_TEMPERATURE,
}
pprint(HPARAMS)

{'batch_size': 5,
 'common_layers': [16, 8],
 'eval_batch_size': 1,
 'feedback_model': 2,
 'global_layers': [64, 32, 16],
 'learning_rate': 0.005,
 'logits_temperature': 1.0,
 'model_type': 'Ranking',
 'network_type': 'dotproduct',
 'num_itmes': 5,
 'num_slots': 2,
 'penalty_mixture': 1.0,
 'per_arm_layers': [64, 32, 16],
 'policy_type': <RankingPolicyType.COSINE_DISTANCE: 1>}


In [56]:
if NETWORK_TYPE == 'commontower':
    scoring_network = global_and_arm_feature_network.create_feed_forward_common_tower_network(
        observation_spec = observation_spec, 
        global_layers = GLOBAL_LAYERS, 
        arm_layers = ARM_LAYERS, 
        common_layers = COMMON_LAYERS,
        # output_dim = output_dim,
    )
    
elif NETWORK_TYPE == 'dotproduct':
    scoring_network = global_and_arm_feature_network.create_feed_forward_dot_product_network(
        observation_spec = observation_spec, 
        global_layers = GLOBAL_LAYERS, 
        arm_layers = ARM_LAYERS
    )
    
if NETWORK_TYPE:
    print(f"Network: {scoring_network.name}")

Network: GlobalAndArmDotProductNetwork


## Define Agent 

In [57]:
rank_agent = ranking_agent.RankingAgent(
    time_step_spec=time_step_spec,
    action_spec=action_spec,
    scoring_network=scoring_network,
    optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=HPARAMS['learning_rate']),
    feedback_model=feedback_model,
    policy_type=HPARAMS['policy_type'],
    logits_temperature=HPARAMS['logits_temperature'],
    penalty_mixture_coefficient=HPARAMS['penalty_mixture'],
    summarize_grads_and_vars=True
)

rank_agent

In [58]:
rank_agent.action_spec

BoundedTensorSpec(shape=(2,), dtype=tf.int32, name='action_spec', minimum=array(0, dtype=int32), maximum=array(4, dtype=int32))

In [59]:
rank_agent.time_step_spec

_TupleWrapper(TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': DictWrapper({'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None), 'per_arm': TensorSpec(shape=(5, 5), dtype=tf.float32, name=None)}),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')}))

In [60]:
rank_agent.training_data_spec

_TupleWrapper(Trajectory(
{'action': (),
 'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'next_step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type'),
 'observation': DictWrapper({'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None), 'per_arm': TensorSpec(shape=(2, 5), dtype=tf.float32, name=None)}),
 'policy_info': (),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')}))

### Reward function

* `_create_ranking_reward_features` - [src](https://source.corp.google.com/piper///depot/google3/learning/smartchoices/training/models/agent_factory_test.py?q=%22ranking_model_factory%22&sq=package:piper%20file:%2F%2Fdepot%2Fgoogle3%20-file:google3%2Fexperimental&start=1)

* `_get_rewards_from_arm_features` 

#### tmp - debugging

In [71]:
def _get_ranking_rewards_sv(x):
    """Calculates reward for the actions."""
    
    # rating_scores_list = []
    ratings_list = x["user_rating"] #[0]
    indices = tf.argsort(ratings_list, direction="DESCENDING")
    
    feedback = tf.gather(ratings_list, indices, batch_dims=-1) #.numpy()
    
    feedback = tf.math.top_k(feedback, k=HPARAMS['num_slots']).values
    
    return feedback

In [72]:
test_ratings = _get_ranking_rewards_sv(data)
test_ratings

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[5., 4.]], dtype=float32)>

In [74]:
# _get_rewards_from_arm_features

In [97]:
def _rank_trajectory_fn(element): # hparams
    """Converts a dataset element into a trajectory."""
    # global_features = _get_global_context_features(element)
    # arm_features = _get_per_arm_features(element)
    
    global_features = _get_global_context_features(element)
    arm_features = _get_per_arm_features(element)
    
    # obs spec
    observation = {
        bandit_spec_utils.GLOBAL_FEATURE_KEY:
            # train_utils._add_outer_dimension(global_features),
            global_features,
        bandit_spec_utils.PER_ARM_FEATURE_KEY: 
            # train_utils._add_outer_dimension(arm_features)
            arm_features
    }
    
    # reward = element['user_rating']
    ranking_rewards = _get_ranking_rewards_sv(element)
    # ranking_rewards = train_utils._add_outer_dimension(_get_ranking_rewards_sv(element))
    
    policy_info = policy_utilities.PolicyInfo(
        # predicted_rewards_mean=tf.zeros([HPARAMS['batch_size'], 1, HPARAMS['num_slots']]),
        predicted_rewards_mean=HPARAMS['num_slots'],
        # Ranking policies do not use the `bandit_policy_type` info field
        # bandit_policy_type=tf.zeros([HPARAMS['batch_size'], 1, 1], dtype=tf.int32),
    )
    
    action = np.zeros((HPARAMS['batch_size']), dtype=np.float32)
    # action=tf.zeros_like(ranking_rewards, dtype=tf.int32)
    
    discount = np.zeros((HPARAMS['batch_size']), dtype=np.float32)
    # discount=tf.zeros_like(ranking_rewards, dtype=tf.int32)
    return trajectory.single_step(
        observation=observation,
        action=action,
        policy_info=(), #policy_info,
        reward=ranking_rewards,
        discount=discount
    )


In [98]:
for x in train_dataset.batch(HPARAMS['batch_size']).take(1):
    test_traj = _rank_trajectory_fn(x)
    
test_traj

concat_embeddings shape: (5, 64)


Trajectory(
{'action': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 'discount': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>,
 'next_step_type': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([2, 2, 2, 2, 2], dtype=int32)>,
 'observation': {'global': <tf.Tensor: shape=(5, 1, 1, 64), dtype=float32, numpy=
array([[[[-9.46198776e-03, -7.79101998e-03,  2.25365646e-02,
          -2.81036031e-02, -4.90761884e-02, -1.11315399e-03,
           1.59021355e-02, -1.03365071e-02, -2.27282289e-02,
          -3.61915119e-02, -2.71073114e-02, -2.96614524e-02,
          -1.31333992e-03, -4.35202979e-02, -5.28260320e-03,
          -3.77968065e-02, -4.00480628e-02, -4.46952842e-02,
           3.35851945e-02, -3.44019420e-02,  2.00893991e-02,
           7.74459913e-03,  4.80566062e-02,  4.01390083e-02,
           2.09614523e-02, -7.09068775e-03, -1.05105713e-03,
           1.18973106e-03,  3.06961201e-02, -4.640710

In [99]:
print(f"test_traj.action.shape: {test_traj.action.shape}") 

test_traj.action.shape: (5,)


In [100]:
print(f"test_traj.discount.shape: {test_traj.discount.shape}") 

test_traj.discount.shape: (5,)


In [101]:
print(f"test_traj.reward.shape: {test_traj.reward.shape}") 

test_traj.reward.shape: (5, 2)


In [102]:
print(f"test_traj.observation.shape: {test_traj.observation['global'].shape}") 

test_traj.observation.shape: (5, 1, 1, 64)


In [103]:
print(f"test_traj.observation.shape: {test_traj.observation['per_arm'].shape}") 

test_traj.observation.shape: (5, 1, 5, 64)


In [104]:
observation_spec

{'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None),
 'per_arm': TensorSpec(shape=(5, 5), dtype=tf.float32, name=None)}

In [105]:
expected_num_actions = action_spec.maximum - action_spec.minimum + 1
print(f"expected_num_actions: {expected_num_actions}")

predicted_rewards_mean = tensor_spec.TensorSpec([expected_num_actions])
print(f"predicted_rewards_mean: {predicted_rewards_mean}")

expected_num_actions: 5
predicted_rewards_mean: TensorSpec(shape=(5,), dtype=tf.float32, name=None)


# Train Ranking Agent

In [106]:
EXPERIMENT_NAME   = f'local-ranker-{PREFIX}'

# new experiment
invoke_time       = time.strftime("%Y%m%d-%H%M%S")
RUN_NAME          = f'run-{invoke_time}'

BASE_OUTPUT_DIR   = f'{BUCKET_URI}/{EXPERIMENT_NAME}/{RUN_NAME}'
LOG_DIR           = f"{BASE_OUTPUT_DIR}/logs"
ROOT_DIR          = f"{BASE_OUTPUT_DIR}/root"       # Root directory for writing logs/summaries/checkpoints.
ARTIFACTS_DIR     = f"{BASE_OUTPUT_DIR}/artifacts"  # Where the trained model will be saved and restored.

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    experiment=EXPERIMENT_NAME
)

print(f"EXPERIMENT_NAME   : {EXPERIMENT_NAME}")
print(f"RUN_NAME          : {RUN_NAME}\n")
print(f"BASE_OUTPUT_DIR   : {BASE_OUTPUT_DIR}")
print(f"LOG_DIR           : {LOG_DIR}")
print(f"ROOT_DIR          : {ROOT_DIR}")
print(f"ARTIFACTS_DIR     : {ARTIFACTS_DIR}")

EXPERIMENT_NAME   : local-ranker-rec-bandits-v1
RUN_NAME          : run-20231003-203431

BASE_OUTPUT_DIR   : gs://rec-bandits-v1-hybrid-vertex-bucket/local-ranker-rec-bandits-v1/run-20231003-203431
LOG_DIR           : gs://rec-bandits-v1-hybrid-vertex-bucket/local-ranker-rec-bandits-v1/run-20231003-203431/logs
ROOT_DIR          : gs://rec-bandits-v1-hybrid-vertex-bucket/local-ranker-rec-bandits-v1/run-20231003-203431/root
ARTIFACTS_DIR     : gs://rec-bandits-v1-hybrid-vertex-bucket/local-ranker-rec-bandits-v1/run-20231003-203431/artifacts


In [107]:
import collections

from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import tf_policy
from tf_agents.trajectories import time_step as ts
from tf_agents.policies import policy_saver
from tf_agents.metrics import export_utils
from tf_agents.metrics import tf_metrics
from tf_agents.eval import metric_utils
from tf_agents.utils import common

import tf_agents

print(f"tf_agents version: {tf_agents.__version__}")
print(f"tensorflow version: {tf.__version__}")

tf_agents version: 0.17.0
tensorflow version: 2.13.0


### logs & checkpoints

In [108]:
global_step = tf.compat.v1.train.get_or_create_global_step()

rank_agent.initialize()
print(f'agent: {rank_agent.name}')
print(f'agent: {rank_agent.policy.name}')

agent: ranking_agent
agent: penalize_cosine_distance_ranking_policy


In [109]:
# ====================================================
# TB summary writer
# ====================================================
train_summary_writer = tf.compat.v2.summary.create_file_writer(
    f"{LOG_DIR}/train", flush_millis=10 * 1000
)
# train_summary_writer.set_as_default()

# eval_summary_writer = tf.compat.v2.summary.create_file_writer(
#     f"{LOG_DIR}/eval", flush_millis=10 * 1000
# )
# ====================================================
# metrics
# ====================================================
# `step_metric` records the number of individual rounds of bandit interaction;
# that is, (number of trajectories) * batch_size
step_metric = tf_metrics.EnvironmentSteps()

if feedback_model == ranking_environment.FeedbackModel.SCORE_VECTOR:
    reward_metric = tf_metrics.AverageReturnMetric(
        batch_size=HPARAMS['batch_size'],
        buffer_size=20
    )
else:
    reward_metric = tf_metrics.AverageReturnMultiMetric(
        reward_spec=environment.reward_spec(),
        batch_size=HPARAMS['batch_size'],
        buffer_size=20
    )
    
metrics = [reward_metric]

pprint(f"metrics: {metrics}")

# ====================================================
# get checkpoint manager
# ====================================================
CHKPOINT_DIR = f"{ROOT_DIR}/chkpoint"
print(f"setting checkpoint_manager: {CHKPOINT_DIR}")

checkpoint_manager = train_utils.restore_and_get_checkpoint_manager(
    root_dir=CHKPOINT_DIR, 
    agent=rank_agent, 
    metrics=metrics, 
    step_metric=step_metric
)

('metrics: [<tf_agents.metrics.tf_metrics.AverageReturnMetric object at '
 '0x7f584696ffa0>]')
setting checkpoint_manager: gs://rec-bandits-v1-hybrid-vertex-bucket/local-ranker-rec-bandits-v1/run-20231003-203431/root/chkpoint


In [110]:
# # ====================================================
# # policy saver
# # ====================================================
# saver = policy_saver.PolicySaver(
#     policy = rank_agent.policy, 
#     # train_step=global_step
# )

In [111]:
policy = rank_agent.policy
isinstance(policy, tf_policy.TFPolicy)

True

## Train config

In [112]:
TRAIN_DATA_SIZE = 900          # len(list(train_dataset))
NUM_TRAIN_STEPS = 180            # TRAIN_DATA_SIZE // HPARAMS['batch_size']

EVAL_DATA_SIZE  = 900          # len(list(val_dataset))
NUM_EVAL_STEPS  = 100           # EVAL_DATA_SIZE // HPARAMS['eval_batch_size']

CHKPT_INTERVAL  = NUM_TRAIN_STEPS # // 5
LOG_INTERVAL    = 10
# EVAL_INTERVAL = NUM_TRAIN_STEPS // 2

print(f"TRAIN_DATA_SIZE : {TRAIN_DATA_SIZE}")
print(f"NUM_TRAIN_STEPS : {NUM_TRAIN_STEPS}")
print(f"EVAL_DATA_SIZE : {EVAL_DATA_SIZE}")
print(f"NUM_EVAL_STEPS : {NUM_EVAL_STEPS}")
print(f"CHKPT_INTERVAL: {CHKPT_INTERVAL}")
print(f"LOG_INTERVAL : {LOG_INTERVAL}")
# print(f"EVAL_INTERVAL : {EVAL_INTERVAL}")

TRAIN_DATA_SIZE : 900
NUM_TRAIN_STEPS : 180
EVAL_DATA_SIZE : 900
NUM_EVAL_STEPS : 100
CHKPT_INTERVAL: 180
LOG_INTERVAL : 10


In [113]:
train_dataset

<_ParallelMapDataset element_spec={'movie_genres': TensorSpec(shape=(5,), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(5,), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(5,), dtype=tf.float32, name=None)}>

In [114]:
# train data
# train_ds_iterator = iter(train_dataset.batch(HPARAMS['batch_size']))
train_ds_iterator = iter(train_dataset)

# eval dataset
eval_ds = val_dataset.batch(HPARAMS["eval_batch_size"])

if NUM_EVAL_STEPS > 0:
    eval_ds = eval_ds.take(NUM_EVAL_STEPS)

# eval_ds = eval_ds.prefetch(tf.data.AUTOTUNE)

eval_ds

<_TakeDataset element_spec={'movie_genres': TensorSpec(shape=(None, 5), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(None, 5), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'user_rating': TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)}>

In [115]:
list_o_loss = []

rank_agent.train_step_counter.assign(0)

print(f"starting train loop...")
start_time = time.time()

# for i in tqdm(range(NUM_TRAIN_STEPS)):
for i in range(NUM_TRAIN_STEPS):
    
    with train_summary_writer.as_default():

        data = next(train_ds_iterator)
        trajectories = _rank_trajectory_fn(data)

        # All tensors in experience must be shaped [batch, time, ...] 
        step = rank_agent.train_step_counter.numpy()
        loss = rank_agent.train(experience=trajectories)
        list_o_loss.append(loss.loss.numpy())
        
        train_utils._export_metrics_and_summaries(
            step=i, 
            metrics=metrics
        )
        
        # print step loss
        if step % LOG_INTERVAL == 0:
            print(
                'step = {0}: train loss = {1}'.format(
                    step, round(loss.loss.numpy(), 2)
                )
            )

        if i > 0 and i % CHKPT_INTERVAL == 0:
            saver.save(os.path.join(CHKPOINT_DIR, 'policy_%d' % step_metric.result()))
            print(f"saved policy to: {CHKPOINT_DIR}")
            
runtime_mins = int((time.time() - start_time) / 60)
print(f"train runtime_mins: {runtime_mins}")

starting train loop...


InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} params must be at least 1 dimensional [Op:GatherV2] name: 

In [94]:
data

{'movie_genres': <tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 3,  0,  0,  0, 19])>,
 'movie_id': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'94', b'245', b'403', b'50', b'470'], dtype=object)>,
 'user_id': <tf.Tensor: shape=(), dtype=string, numpy=b'346'>,
 'user_rating': <tf.Tensor: shape=(5,), dtype=float32, numpy=array([3., 4., 3., 5., 3.], dtype=float32)>}

### pre-training eval loop

In [312]:
from src.perarm_features import eval_perarm as eval_perarm

In [313]:
# dummy_arm = tf.zeros([HPARAMS['eval_batch_size'], PER_ARM_DIM], dtype=tf.float32)
# dummy_arm

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[0., 0., 0., 0., 0.]], dtype=float32)>

In [315]:
# serving_trajectory_fn(experience.observation)

In [317]:
for x in eval_ds:

    filter_mask = None
    # get feature tensors

#     global_feat_infer = _get_global_context_features(x)
#     arm_feat_infer = _get_per_arm_features(x)
    
#     feature = {'global': global_feat_infer, 'per_arm': arm_feat_infer}

#     rewards = x['user_rating']
    
    # reshape arm features
    # arm_feat_infer = tf.reshape(arm_feat_infer, [HPARAMS['eval_batch_size'], PER_ARM_DIM])
    # concat_arm = tf.concat([arm_feat_infer, dummy_arm], axis=0)
    
    # trajectory_step = train_utils._get_eval_step(feature, rewards.numpy()[0])
    trajectory_step = _rank_trajectory_fn(x)
    
    break

concat_embeddings shape: (5, 64)
concat_embeddings shape: (5, 64)


In [321]:
# rewards.numpy()[0]

In [320]:
# feature

In [322]:
# arm_feat_infer

In [ ]:
# ====================================================
# Evaluate the agent's policy once before training
# ====================================================
# Reset the train step
rank_agent.train_step_counter.assign(0)

pre_policy_tf = py_tf_eager_policy.PyTFEagerPolicy(rank_agent.policy, use_tf_function=True)

print(f"evaluating pre-trained Agent...")
start_time = time.time()

pre_val_loss, pre_preds, pre_tr_rewards = eval_perarm._run_bandit_eval(
    policy = pre_policy_tf,
    data = eval_ds,
    eval_batch_size = HPARAMS['eval_batch_size'],
    per_arm_dim = PER_ARM_DIM,
    global_dim = GLOBAL_DIM,
    vocab_dict = vocab_dict,
    num_oov_buckets = NUM_OOV_BUCKETS,
    global_emb_size = GLOBAL_EMBEDDING_SIZE,
    mv_emb_size = MV_EMBEDDING_SIZE,
)

runtime_mins = int((time.time() - start_time) / 60)
print(f"pre-train val_loss     : {pre_val_loss}")
print(f"pre-train eval runtime : {runtime_mins}")
# ====================================================
# train loop
# ====================================================
print(f"starting train loop...")
start_time = time.time()

### Ranking Task

In [ ]:
task = tfrs.tasks.Ranking(
      loss=loss,
      metrics=[
        tfr.keras.metrics.NDCGMetric(name="ndcg_metric"),
        tf.keras.metrics.RootMeanSquaredError()
      ]
    )

In [21]:
from src.perarm_features import emb_features as emb_features

embs = emb_features.EmbeddingModel(
    vocab_dict = vocab_dict,
    num_oov_buckets = NUM_OOV_BUCKETS,
    global_emb_size = GLOBAL_EMBEDDING_SIZE,
    mv_emb_size = MV_EMBEDDING_SIZE,
)

embs

In [23]:
test_globals = embs._get_global_context_features(data)
 
GLOBAL_DIM = test_globals.shape[1]   

# test_globals = embs._get_global_context_features_v2(data)
# GLOBAL_DIM, _ = test_globals #.shape[1]  
# GLOBAL_DIM = GLOBAL_DIM.shape[1]

print(f"GLOBAL_DIM: {GLOBAL_DIM}")
test_globals

GLOBAL_DIM: 64


array([[ 0.02324566,  0.03847075,  0.0009382 ,  0.01037087, -0.0168501 ,
         0.02229491, -0.00090174, -0.03733226,  0.04767699, -0.04324825,
         0.03009412, -0.00325273,  0.00937269,  0.01162167,  0.04858048,
        -0.03228705, -0.00880662,  0.00480435,  0.0266616 , -0.01046257,
        -0.01173887, -0.04642478,  0.0118764 , -0.03422302,  0.0312058 ,
        -0.02052242,  0.00459113, -0.0279305 , -0.00947496, -0.00711267,
         0.04388683, -0.02854126, -0.02312779, -0.04988689,  0.03593904,
        -0.02224693, -0.01872089,  0.0384705 ,  0.0432454 ,  0.01695739,
        -0.02577996, -0.01301978, -0.01603407,  0.02314747, -0.01779956,
         0.03170073,  0.00098064, -0.0470944 , -0.02522329,  0.01904691,
         0.00721593,  0.0179752 ,  0.00104992,  0.00527094, -0.04577625,
        -0.02623909, -0.04071148, -0.01163318, -0.02864906, -0.00920088,
        -0.00450635, -0.0427685 ,  0.04297205,  0.04952253]],
      dtype=float32)

In [24]:
test_arms = embs._get_per_arm_features(data)

PER_ARM_DIM = test_arms.shape[1]            
# shape checks out at batch_dim, nactions, arm feats
print(f"PER_ARM_DIM: {PER_ARM_DIM}")

test_arms

PER_ARM_DIM: 64


array([[ 0.02343505,  0.0119458 ,  0.02910166, -0.03252503,  0.00803228,
         0.03554758,  0.01453496, -0.01542882, -0.01640511, -0.03279047,
         0.01140917, -0.03903098,  0.03479561,  0.0006261 ,  0.01132431,
         0.0158616 , -0.0487823 ,  0.04276118,  0.01437578,  0.02662202,
         0.01141692,  0.03396836,  0.04457219,  0.01392839, -0.04691635,
         0.01952115,  0.00976838,  0.04489179,  0.04724428, -0.02655888,
         0.04153249, -0.00589923,  0.01623068,  0.00170443, -0.00777126,
        -0.0101654 ,  0.01170176, -0.01047592, -0.04902004,  0.01853789,
        -0.04450876,  0.0009967 , -0.01994429,  0.02006539, -0.02091893,
         0.01542037, -0.01819051, -0.03646524, -0.02382381,  0.01859686,
         0.03186891,  0.01153149, -0.03137907, -0.01640514,  0.01935514,
        -0.03642275, -0.02170839, -0.04142233,  0.01269498, -0.0153484 ,
        -0.01838543, -0.00584408, -0.01540409,  0.03383145]],
      dtype=float32)

### Create a moive lookup Table

In [25]:
movie_lookup_table = {
    'id': [],
    'movie_features': [],
    # 'movie_title': [],
    'movie_genres': []
}
    
iterator = iter(train_dataset.batch(1000))

for data in iterator:
    embs._get_per_arm_features(data)
    movie_lookup_table['id'].extend(data['movie_id'].numpy())
    # movie_lookup_table['movie_title'].extend(data['movie_title'].numpy())
    movie_lookup_table['movie_genres'].extend(data['movie_genres'].numpy())
    movie_lookup_table['movie_features'].extend(embs._get_per_arm_features(data))
    
#fix string ids to integers for random lookup later
movie_lookup_table['id'] = [int(x) for x in movie_lookup_table['id']]

In [26]:
movie_lookup_table = pd.DataFrame(movie_lookup_table)
movie_lookup_table.set_index(['id'])

unique_table = movie_lookup_table.groupby(['id'])[
    [
        'movie_features', 
        # 'movie_title', 
        'movie_genres'
    ]
].first().reset_index() #resetting index to get consecutive counts from min-max (no gaps)
# unique_table = unique_table['movie_features']
MAX_ARM_ID = len(unique_table)-1
MIN_ARM_ID = 0

# unique_table
# print(f"Max movie id is: {MAX_ARM_ID} \nMin movie id is: {MIN_ARM_ID}")

In [27]:
unique_table.iloc[2,:]['movie_features'] #example of getting a ra movie

array([ 0.02495834, -0.01222584,  0.02913784,  0.00382224, -0.00345637,
       -0.02219919,  0.03300631,  0.02205736, -0.03309649, -0.02158695,
        0.04930801, -0.00045602, -0.00740384,  0.04815162,  0.00566232,
       -0.01206527, -0.04011983,  0.00283178,  0.03639659, -0.04393387,
        0.02940616,  0.02252943, -0.04776486, -0.00800429,  0.02306003,
        0.03619469, -0.02831327, -0.01686976,  0.04269502, -0.02647526,
        0.00538781,  0.04991165,  0.02267591,  0.01639385, -0.02517967,
        0.01744567, -0.03633409, -0.01866313,  0.01826422,  0.04594373,
        0.01793351, -0.04969008,  0.03802439, -0.00140975,  0.02682305,
       -0.0081509 , -0.0046325 ,  0.01475661,  0.03066139, -0.0255517 ,
        0.00646546,  0.00037592, -0.0198789 ,  0.0383074 , -0.04167994,
       -0.00177239, -0.01200641, -0.00897124,  0.03143324, -0.03409   ,
       -0.01962967, -0.0414306 , -0.04632906,  0.00900277], dtype=float32)

In [28]:
def get_random_arm_features(movie_id):
    movie_info = unique_table.iloc[movie_id]
    tensor = tf.constant(movie_info['movie_features'], dtype=tf.float32)
    return tf.reshape(tensor, [1, tensor.shape[0]]), [movie_info['movie_genres']],
                                                     # movie_info['movie_genres']]

get_random_arm_features(222)

(<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
 array([[ 0.02758646, -0.02364726,  0.00326383, -0.00670724, -0.02581365,
          0.02811625, -0.02409014,  0.02277763,  0.03388413,  0.01355955,
         -0.04816724, -0.04114227,  0.01485746,  0.01675477,  0.02701057,
         -0.018884  ,  0.00393082, -0.01786947,  0.01814145,  0.00791826,
         -0.03671928,  0.02370023, -0.03795796,  0.00273353,  0.01422632,
          0.00815446,  0.03875173, -0.00820974,  0.03643649, -0.04565177,
         -0.01958307, -0.00754724,  0.01623068,  0.00170443, -0.00777126,
         -0.0101654 ,  0.01170176, -0.01047592, -0.04902004,  0.01853789,
         -0.04450876,  0.0009967 , -0.01994429,  0.02006539, -0.02091893,
          0.01542037, -0.01819051, -0.03646524, -0.02382381,  0.01859686,
          0.03186891,  0.01153149, -0.03137907, -0.01640514,  0.01935514,
         -0.03642275, -0.02170839, -0.04142233,  0.01269498, -0.0153484 ,
         -0.01838543, -0.00584408, -0.01540409,  0.03383145]],

In [29]:
def get_random_set_of_arm_features(n_actions):
    random_arm_ids = list(np.random.randint(MIN_ARM_ID, MAX_ARM_ID, n_actions))
    features = [get_random_arm_features(x) for x in random_arm_ids]
    just_features = [x[0] for x in features]
    movie_info = [x[1] for x in features]
    return tf.concat(just_features, axis=0), movie_info

In [30]:
#NEW - there's a tuple returned with the movies we will use for PALM!
get_random_set_of_arm_features(n_actions=2)[0]

<tf.Tensor: shape=(2, 64), dtype=float32, numpy=
array([[ 0.03378742,  0.01706399,  0.03097352, -0.03447683,  0.00684074,
        -0.02467006, -0.03587176, -0.00194854,  0.00817044,  0.00660542,
         0.02049113,  0.02787698, -0.01884164,  0.0250637 , -0.02671324,
         0.00575078,  0.00014221,  0.02990893, -0.01085119, -0.03606417,
         0.02069755, -0.02867526,  0.03644859,  0.02053731, -0.03130388,
         0.02352465,  0.04758057, -0.02958646, -0.02110904, -0.02467135,
        -0.04586595,  0.00947623,  0.03001981,  0.02514908,  0.03943818,
         0.03974951, -0.02048717,  0.00589327, -0.01928915, -0.01875315,
         0.03245391,  0.02953072, -0.01104504, -0.04738101, -0.01941533,
        -0.03232567, -0.04955465,  0.04904519, -0.01213044,  0.01257041,
         0.0311668 , -0.03899008, -0.0284349 ,  0.02622536,  0.02824564,
         0.00210474,  0.03348925, -0.00880535,  0.04751078,  0.03034684,
         0.02982635, -0.00188724, -0.0043234 , -0.0364937 ],
       [ 0.001

In [31]:
# ### Look at the raw input features to format a good prompt for ranking movies
# NUM_ACTIONS = 5
# batch_size = 8
# iterator = iter(train_dataset.batch(batch_size))
# data = next(iterator)

# _, user_info = embs._get_global_context_features_v2(data)
# ###NEW - we are getting the arm features here
# _, movie_info = get_random_set_of_arm_features(n_actions=NUM_ACTIONS)

# print(user_info, movie_info)

In [32]:
from datetime import datetime
dt = datetime.utcfromtimestamp(885409515)
dt.ctime()

'Wed Jan 21 19:05:15 1998'

In [33]:
# movie_info

## Ranking Bandit

In [32]:
BATCH_SIZE      = 128
EVAL_BATCH_SIZE = 1

NUM_ACTIONS = 2
NUM_ITEMS  = 50
NUM_SLOTS  = 3 

print(f"BATCH_SIZE      : {BATCH_SIZE}")
print(f"EVAL_BATCH_SIZE : {EVAL_BATCH_SIZE}")
print(f"NUM_ACTIONS     : {NUM_ACTIONS}")
print(f"GLOBAL_DIM      : {GLOBAL_DIM}")
print(f"PER_ARM_DIM     : {PER_ARM_DIM}")

BATCH_SIZE      : 128
EVAL_BATCH_SIZE : 1
NUM_ACTIONS     : 2
GLOBAL_DIM      : 64
PER_ARM_DIM     : 64


## specs

### Observation Spec

**example config:**
```
global_dim = 9  #@param{ type: "integer"}
item_dim   = 11  #@param{ type: "integer"}
num_items  = 50 #@param{ type: "integer"}
num_slots  = 3  #@param{ type: "integer"}
```
**example obs sepc:**

```
{'global': TensorSpec(shape=(9,), dtype=tf.float32, name=None),
 'per_arm': TensorSpec(shape=(50, 11), dtype=tf.float32, name=None)}
 ```

In [33]:
observation_spec = {
    'global': tf.TensorSpec([GLOBAL_DIM], tf.float32),
    'per_arm': tf.TensorSpec([NUM_ACTIONS, PER_ARM_DIM], tf.float32) #excluding action dim here
}
observation_spec

{'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None),
 'per_arm': TensorSpec(shape=(2, 64), dtype=tf.float32, name=None)}

### Action Spec

```
BoundedTensorSpec(shape=(3,), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(49, dtype=int32))

```

In [34]:
action_spec = tensor_spec.BoundedTensorSpec(
    shape=[], 
    dtype=tf.int32,
    minimum=tf.constant(0),            
    maximum=NUM_SLOTS-1, # n degrees of freedom and will dictate the expected mean reward spec shape
    name="action_spec"
)

action_spec

BoundedTensorSpec(shape=(), dtype=tf.int32, name='action_spec', minimum=array(0, dtype=int32), maximum=array(2, dtype=int32))

### TimeStep Spec

```
TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': {'global': TensorSpec(shape=(9,), dtype=tf.float32, name=None),
                 'per_arm': TensorSpec(shape=(50, 11), dtype=tf.float32, name=None)},
 'reward': {'chosen_index': BoundedTensorSpec(shape=(), dtype=tf.int32, name='chosen_index', minimum=array(0, dtype=int32), maximum=array(3, dtype=int32)),
            'chosen_value': TensorSpec(shape=(), dtype=tf.float32, name='chosen_value')},
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})
 ```

In [35]:
time_step_spec = ts.time_step_spec(observation_spec)#, reward_spec=tf.TensorSpec([1, NUM_ACTIONS]))
time_step_spec

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': {'global': TensorSpec(shape=(64,), dtype=tf.float32, name=None),
                 'per_arm': TensorSpec(shape=(2, 64), dtype=tf.float32, name=None)},
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

In [ ]:
feedback_model = ranking_environment.FeedbackModel

In [ ]:
from src.perarm_features import ranking_bandit_policy
# policy = ranking_bandit_policy.GenLinearRankingBanditPolicy(

In [ ]:
from src.perarm_features import agent_factory as agent_factory
ranking_bandit_agent = agent_factory.GenLinearRankingBanditAgent(XXX)